In [1]:
%load_ext autoreload
%load_ext memory_profiler 
%load_ext dotenv
%autoreload 2
%dotenv

In [28]:
import io
import h5py
import pandas as pd
import imageio.v3 as iio
import matplotlib.pyplot as plt
from pathlib import Path
from geovision.data.imagenet import ImagenetETL, ImagenetHDF5Classification
from torch.utils.data import DataLoader
from tqdm import tqdm

ds = ImagenetHDF5Classification('val')
dl = DataLoader(ds, 256, shuffle = True, num_workers=3, pin_memory=True, prefetch_factor=128)
for batch in tqdm(dl, total = len(dl)):
    images, label_idxs, df_idxs = batch

100%|██████████| 251/251 [01:48<00:00,  2.31it/s]


In [3]:
import logging
from lightning import Trainer
from geovision.experiment.config import ExperimentConfig
from geovision.models.interfaces import ClassificationModule
from geovision.data import ImageDatasetDataModule

from geovision.io.local import FileSystemIO as fs
from geovision.experiment.loggers import (
    get_csv_logger, 
    get_ckpt_logger,
    get_classification_logger
)

In [7]:
config = ExperimentConfig.from_yaml("config.yaml")
datamodule = ImageDatasetDataModule(config.dataset_constructor, config.dataset_config, config.dataloader_config)

logging.basicConfig(
    filename = f"{fs.get_new_dir(config.experiments_dir, "logs")}/logfile.log",
    filemode = "a",
    format = "%(asctime)s : %(name)s : %(levelname)s : %(message)s",
    level = logging.INFO
)

loggers: list = list()
loggers.append(csv_logger := get_csv_logger(config))

callbacks: list = list()
#callbacks.append(ckpt_logger := get_ckpt_logger(config))
callbacks.append(metrics_logger := get_classification_logger(config))

In [8]:
config = ExperimentConfig.from_yaml("config.yaml")
model = ClassificationModule(
    model_config=config.model_config,
    criterion_constructor=config.criterion_constructor, 
    criterion_params=config.criterion_params, 
    optimizer_constructor=config.optimizer_constructor,
    optimizer_params=config.optimizer_params,
    lr_scheduler_constructor=config.scheduler_constructor,
    lr_scheduler_params=config.scheduler_params,
    warmup_scheduler_constructor=config.warmup_scheduler_constructor,
    warmup_scheduler_params=config.warmup_scheduler_params,
    warmup_steps=config.warmup_steps,
    scheduler_config_params=config.scheduler_config_params
)
trainer = Trainer(logger = loggers, callbacks = callbacks, **config.trainer_params)
trainer.fit(
    model = model, 
    datamodule = datamodule, 
    ckpt_path = config.ckpt_path 
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory /home/sambhav/experiments/imagenet/testing_dataloading exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!


Empty DataFrame
Columns: [ckpt_path, epoch, step, version]
Index: []


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | encoder   | ResNetFeatureExtractor | 11.2 M | train
1 | decoder   | Linear                 | 513 K  | train
2 | criterion | CrossEntropyLoss       | 0      | train
-------------------------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.758    Total estimated model params size (MB)
83        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined